
# Marching Cubes

Marching cubes is an algorithm to extract a 2D surface mesh from a 3D volume.
This can be conceptualized as a 3D generalization of isolines on topographical
or weather maps. It works by iterating across the volume, looking for regions
which cross the level of interest. If such regions are found, triangulations
are generated and added to an output mesh. The final result is a set of
vertices and a set of triangular faces.

The algorithm requires a data volume and an isosurface value. For example, in
CT imaging Hounsfield units of +700 to +3000 represent bone. So, one potential
input would be a reconstructed CT set of data and the value +700, to extract
a mesh for regions of bone or bone-like density.

This implementation also works correctly on anisotropic datasets, where the
voxel spacing is not equal for every spatial dimension, through use of the
`spacing` kwarg.


In [1]:
pip install pymeshlab

     |████████████████████████████████| 42.3 MB 31 kB/s  eta 0:00:01    |███                             | 4.1 MB 760 kB/s eta 0:00:51
     |████████████████████████████████| 15.4 MB 747 kB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install scikit-image

     |████▎                           | 4.0 MB 685 kB/s eta 0:00:39^C

ERROR: Operation cancelled by user
Note: you may need to restart the kernel to use updated packages.


In [4]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

from skimage import measure
from skimage.draw import ellipsoid

import pymeshlab
import requests as req

from PIL import Image

import numpy as np


from io import BytesIO 

im = Image.open("myimage.tiff")

image_array = np.array(im)


chunks = 12
step = int(image_array.shape[0]/chunks)

collection = []
for y in range(0,chunks):
    for x in range(0,chunks):
        print(x,y)
        xlim = [x*step,(x+1)*step]
        ylim = [y*step,(y+1)*step]

        # create vectors and floor some of the terrain data
        xrange = np.arange(xlim[0],xlim[1]-1)
        yrange = np.arange(ylim[0],ylim[1]-1)
        xv,yv = np.meshgrid(xrange,yrange)
        ## ensure we divide by factor for better results in meshlab, not as spiky terrain
        height_vals = image_array[xv,yv]/2
        collection.append({"xlim":xlim,"ylim":ylim,"heights":height_vals})
   

0 0
1 0
2 0
3 0
4 0
5 0
6 0
7 0
8 0
9 0
10 0
11 0
0 1
1 1
2 1
3 1
4 1
5 1
6 1
7 1
8 1
9 1
10 1
11 1
0 2
1 2
2 2
3 2
4 2
5 2
6 2
7 2
8 2
9 2
10 2
11 2
0 3
1 3
2 3
3 3
4 3
5 3
6 3
7 3
8 3
9 3
10 3
11 3
0 4
1 4
2 4
3 4
4 4
5 4
6 4
7 4
8 4
9 4
10 4
11 4
0 5
1 5
2 5
3 5
4 5
5 5
6 5
7 5
8 5
9 5
10 5
11 5
0 6
1 6
2 6
3 6
4 6
5 6
6 6
7 6
8 6
9 6
10 6
11 6
0 7
1 7
2 7
3 7
4 7
5 7
6 7
7 7
8 7
9 7
10 7
11 7
0 8
1 8
2 8
3 8
4 8
5 8
6 8
7 8
8 8
9 8
10 8
11 8
0 9
1 9
2 9
3 9
4 9
5 9
6 9
7 9
8 9
9 9
10 9
11 9
0 10
1 10
2 10
3 10
4 10
5 10
6 10
7 10
8 10
9 10
10 10
11 10
0 11
1 11
2 11
3 11
4 11
5 11
6 11
7 11
8 11
9 11
10 11
11 11


In [5]:
import multiprocessing as mp
from multiprocessing import Queue 

res_queue = Queue()

def run_process(q):
    my_count = 0
    
    while not q.empty():
        
        collection_item = q.get()
        
        xlim = collection_item["xlim"]
        ylim = collection_item["ylim"]

        height_vals = collection_item["heights"]
        valrange = [np.min(height_vals),np.max(height_vals)]
        height_vals = np.floor(height_vals - valrange[0]).astype("int16")
        xrange = np.arange(xlim[0],xlim[1]-1)
        yrange = np.arange(ylim[0],ylim[1]-1)
        xv,yv = np.meshgrid(xrange,yrange)


        zeros = np.zeros((xlim[1] - xlim[0],ylim[1]-ylim[0],int(valrange[1] - valrange[0]) + 1),dtype="uint8")

    ## issue that we need to bring back to original 0 - step range
        zeros[xv.flatten()-xlim[0],yv.flatten() - ylim[0],height_vals.flatten()] = 1

        # Use marching cubes to obtain the surface mesh of these ellipsoids
        verts, faces, normals, values = measure.marching_cubes(zeros, 0)
        verts[:,0] = verts[:,0] + xlim[0]
        verts[:,1] = verts[:,1] + ylim[0]
        verts[:,2] = verts[:,2] + valrange[0]
        ms = pymeshlab.MeshSet()

        mesh = pymeshlab.Mesh(verts)
        ms.add_mesh(mesh)
        ## simplify and convert mesh to int16 and add to the res_queue of numpy objects, 
        ms.point_cloud_simplification(samplenum=100000)
        
        data_to_append = ms.current_mesh().vertex_matrix().astype("int16")
        print(data_to_append.shape)
        np.save(f"landscape_data_x{xlim[0]}_y{ylim[0]}.npy",data_to_append)
        
        my_count +=1
        print("processed",my_count)
    
    
## make the queue
q = Queue()
for data in collection:
    q.put(data)
    
threads = []
for data in range(8):
    pthread = mp.Process(target=run_process,args=(q,))
    pthread.start()
    threads.append(pthread)
    
        
for thread in threads:
    thread.join()

(249590, 3)
processed 1
(315275, 3)
processed 1
(253617, 3)
processed 1
(329184, 3)
processed 1
(264026, 3)
processed 2
(191850, 3)
processed 2
(308333, 3)
processed 2
(224597, 3)
processed 2
(267641, 3)
processed 3
(206953, 3)
processed 3
(213454, 3)
processed 3
(182976, 3)
processed 3
(331354, 3)
processed 4
(328181, 3)
processed 4
(250970, 3)
processed 4
(249836, 3)
processed 5
(269500, 3)
processed 4
(279096, 3)
processed 5
(272892, 3)
processed 5
(149126, 3)
processed 6
(182141, 3)
processed 5
(252194, 3)
processed 6
(250944, 3)
processed 6
(259539, 3)
processed 6
(199773, 3)
processed 7
(250333, 3)
processed 7
(256320, 3)
processed 7
(253366, 3)
processed 8
(186326, 3)
processed 8
(235736, 3)
processed 7
(252053, 3)
processed 9
(196011, 3)
processed 8
(253521, 3)
processed 9
(251307, 3)
processed 8
(253021, 3)
processed 10
(329768, 3)
processed 10
(329395, 3)
processed 9
(177990, 3)
processed 9
(250930, 3)
processed 11
(206312, 3)
processed 11
(192192, 3)
processed 10
(242478, 3)

In [6]:
import os
numpys = [f for f in os.listdir() if "landscape_data" in f]

In [7]:
327663*3

982989

In [8]:
np.load("landscape_data_x0_y0.npy").shape

(249590, 3)

In [9]:
all_data = np.zeros((1,3))
for f in numpys:
    print(f)
    try:
        all_data = np.vstack((all_data,np.load(f)))
    except Exception as e:
        print(e)

landscape_data_x0_y0.npy
landscape_data_x0_y1802.npy
landscape_data_x0_y2703.npy
landscape_data_x0_y3604.npy
landscape_data_x0_y4505.npy
landscape_data_x0_y5406.npy
landscape_data_x0_y6307.npy
landscape_data_x0_y7208.npy
landscape_data_x0_y8109.npy
landscape_data_x0_y901.npy
landscape_data_x0_y9010.npy
landscape_data_x0_y9911.npy
landscape_data_x1802_y0.npy
landscape_data_x1802_y1802.npy
landscape_data_x1802_y2703.npy
landscape_data_x1802_y3604.npy
landscape_data_x1802_y4505.npy
landscape_data_x1802_y5406.npy
landscape_data_x1802_y6307.npy
landscape_data_x1802_y7208.npy
landscape_data_x1802_y8109.npy
landscape_data_x1802_y901.npy
landscape_data_x1802_y9010.npy
landscape_data_x1802_y9911.npy
landscape_data_x2703_y0.npy
landscape_data_x2703_y1802.npy
landscape_data_x2703_y2703.npy
landscape_data_x2703_y3604.npy
landscape_data_x2703_y4505.npy
landscape_data_x2703_y5406.npy
landscape_data_x2703_y6307.npy
landscape_data_x2703_y7208.npy
landscape_data_x2703_y8109.npy
landscape_data_x2703_y90

In [10]:
all_data.shape

(33912654, 3)

In [11]:
ms = pymeshlab.MeshSet()
mesh = pymeshlab.Mesh(all_data)
ms.add_mesh(mesh)
ms.save_project("simple_together.mlp")

In [12]:
ms.point_cloud_simplification(samplenum=100000)
ms.save_project("even_simpler.mlp")